In [1]:
import os
import math
from PIL import Image
from PIL import ImageDraw

In [2]:
def get_distance(coor1, coor2):
    return math.sqrt((coor1[0]-coor2[0])**2 + (coor1[1]-coor2[1])**2)

In [6]:
def crop_image(image_path, txt_path, name, count, label_file):
    img = Image.open(''.join((image_path, name, '.jpg')))
    txt = open(''.join((txt_path, name, '.txt')))
    for line in txt.readlines():
        info = line[:-1].split(',')
        label = info[8]
        if info[8] == '###':
            continue
        location = []
        for i in range(4):
            location.append([float(x) for x in info[i*2:i*2+2]])

        x_min = min(location, key=lambda x: x[0])[0]
        x_max = max(location, key=lambda x: x[0])[0]
        y_min = min(location, key=lambda x: x[1])[1]
        y_max = max(location, key=lambda x: x[1])[1]
        if x_max - x_min < 1:
            x_max = x_min + 1
        if y_max - y_min < 1:
            y_max = y_min + 1
        croped_img = img.crop((x_min, y_min, x_max, y_max))

        # 根据较长的边决定
        distance01 = get_distance(location[0], location[1])
        distance12 = get_distance(location[2], location[1])
        if distance01 > distance12:
            if location[0][0] - location[1][0]:
                k = (location[0][1] - location[1][1]) / (location[0][0] - location[1][0])
            else:
                k = float('inf')
            if abs(k) < 0.05 or (distance01 / distance12 < 1.4 and abs(k) > 3):
                angle = 0
            elif abs(k) > 5:
                angle = 90 if distance01 / distance12 > 1.7 and distance01 > 20 else 0
            else:
                angle = math.atan(abs(k)) * 57.3
                angle = -angle if k < 0 else angle
            width = distance01
            height = distance12
            ratio = distance01 / distance12
        else:
            if location[2][0] - location[1][0]:
                k = (location[2][1] - location[1][1]) / (location[2][0] - location[1][0])
            else:
                k = float('inf')
            if abs(k) < 0.05 or (distance12 / distance01 < 1.4 and abs(k) > 3):
                angle = 0
            elif abs(k) > 5:
                angle = 90 if distance12 / distance01 > 1.7 and distance12 > 20 else 0
            else:
                angle = math.atan(abs(k)) * 57.3
                angle = -angle if k < 0 else angle
            width = distance12
            height = distance01
            ratio = distance12 / distance01

        # 旋转切割
        if angle:
            rotated_img = croped_img.rotate(angle, expand=True)
            w, h = rotated_img.size
            croped_img = rotated_img.crop(((w-width)/2, (h-height)/2, (w+width)/2, (h+height)/2))
        croped_img = croped_img.convert('RGB')
        
        # 根据长宽比选择要存入的文件夹
        folder = None
        if ratio > 6.25:
            folder = 8
        elif ratio > 3.125:
            folder = 5
        else:
            folder = 2
        
        # 保存图片
        try:
            croped_img.save('data/image/%d/%d.jpg'%(folder, count,))
        except SystemError as e:
            pass
        # 记录label
        label_file.write(''.join((str(folder), ' ', str(count), ' ', label, ' ', image_path, name, '.jpg', '\n')))
        count += 1
    txt.close()
    return count

In [7]:
data_path = [['ICPR_text_train_part1_20180316/train_1000/image_1000/',
              'ICPR_text_train_part1_20180316/train_1000/txt_1000/'],
             ['ICPR_text_train_part2_20180313/image_9000/',
              'ICPR_text_train_part2_20180313/txt_9000/']]
for i in range(len(data_path)):
    image_path, txt_path = data_path[i]
    file_list = []
    for img in list(os.walk(image_path))[0][2]:
        file_list.append(img[:-4])
    data_path[i].append(file_list)

label_file = open('data/label.txt', 'w')
count = 0
for image_path, txt_path, file_names in data_path:
    for name in file_names:
        count = crop_image(image_path, txt_path, name, count, label_file)
label_file.close()

In [4]:
# for test

def test_image(name, image_path, txt_path):
    img = Image.open(''.join((image_path, name, '.jpg')))
    drawing_img = ImageDraw.Draw(img)
    txt = open(''.join((txt_path, name, '.txt')))
    for line in txt.readlines():
        info = line[:-1].split(',')
        label = info[8]
        if info[8] == '###':
            continue
        location = []
        for i in range(4):
            location.append([float(x) for x in info[i*2:i*2+2]])
        info = [float(x) for x in info[:8]]
        drawing_img.line(info[:4], fill = (255, 100, 0), width=3)
        drawing_img.line(info[2:6], fill = (255, 100, 0), width=3)
        drawing_img.line(info[4:8], fill = (255, 100, 0), width=3)
        drawing_img.line(info[6:8]+info[:2], fill = (255, 100, 0), width=3)

        x_min = min(location, key=lambda x: x[0])[0]
        x_max = max(location, key=lambda x: x[0])[0]
        y_min = min(location, key=lambda x: x[1])[1]
        y_max = max(location, key=lambda x: x[1])[1]
        if x_max - x_min < 1:
            x_max = x_min + 1
        if y_max - y_min < 1:
            y_max = y_min + 1
        croped_img = img.crop((x_min, y_min, x_max, y_max))

        distance01 = get_distance(location[0], location[1])
        distance12 = get_distance(location[2], location[1])
        if distance01 > distance12:
            if location[0][0] - location[1][0]:
                k = (location[0][1] - location[1][1]) / (location[0][0] - location[1][0])
            else:
                k = float('inf')
            if abs(k) < 0.05 or distance01 / distance12 < 1.4:
                angle = 0
            elif abs(k) > 5:
                angle = 90 if distance01 / distance12 > 1.7 and distance01 > 20 else 0
            else:
                angle = math.atan(abs(k)) * 57.3
                angle = -angle if k < 0 else angle
            width = distance01
            height = distance12
        else:
            if location[2][0] - location[1][0]:
                k = (location[2][1] - location[1][1]) / (location[2][0] - location[1][0])
            else:
                k = float('inf')
            if abs(k) < 0.05 or distance12 / distance01 < 1.4:
                angle = 0
            elif abs(k) > 5:
                angle = 90 if distance12 / distance01 > 1.7 and distance12 > 20 else 0
            else:
                angle = math.atan(abs(k)) * 57.3
                angle = -angle if k < 0 else angle
            width = distance12
            height = distance01
        print(k, angle)

        if angle:
            rotated_img = croped_img.rotate(angle, expand=True)
    #         rotated_img.show()
            w, h = rotated_img.size
            croped_img = rotated_img.crop(((w-width)/2, (h-height)/2, (w+width)/2, (h+height)/2))
        try:
            croped_img.show()
        except SystemError as e:
            pass
    img.show()
test_image('TB1_XyUKXXXXXb.XVXXXXXXXXXX_!!0-item_pic.jpg', 'ICPR_text_train_part2_20180313/image_9000/',
              'ICPR_text_train_part2_20180313/txt_9000/')

-0.015299026425591099 0
0.02556177817024144 0
0.005612774675160666 0
0.017241379310344827 0
0.008 0
0.00847457627118644 0
0.02368358727401911 0
33.666666666666664 0
-0.8247126436781611 -39.5157439228285
-0.8268776141116569 -39.589499824431364
-0.7881818181818196 0
-0.3443333333333347 0
-0.8503061224489784 -40.37769169053827
0.041250000000005116 0
-0.7312228429546874 -36.17778890285846
-0.7483272177854506 -36.81121999290611
-0.8461538461538461 -40.2393221754661
-0.8512396694214877 -40.40872301230837
-0.3633265167007498 -19.968896202394017
-0.8286440677966099 -39.649563037165535
-0.4001199760047996 0
-0.8503389830508478 -40.37878447031127
-1.1329999999999985 -48.57155029769288
-0.8313002620403003 -39.73968330933932
0.0 0
-0.8292617208550387 -39.6705399805575
-0.06816632583503768 0
-0.8328109608668399 -39.79083393446008
-0.8472159142519755 -40.274768258672694
-0.7994858611825187 -38.64468797494675
-0.8055555555555558 -38.856236334020316
1.1513636363636357 0
-0.634467618002197 0
-0.5465643